In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [130]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

print(df.shape)
df.head()

(1462, 9)


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [131]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical = list(df.dtypes[(df.dtypes == 'int64') | (df.dtypes == 'float64')].index)

df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [132]:
# Question 1
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [133]:
# Question 2

results = {}

results['interaction_count vs lead_score'] = df[numerical].corrwith(df['interaction_count']).abs()['lead_score']
results['interaction_count vs annual_income'] = df[numerical].corrwith(df['interaction_count']).abs()['annual_income']
results['number_of_courses_viewed vs lead_score'] = df[numerical].corrwith(df['number_of_courses_viewed']).abs()['lead_score']
results['number_of_courses_viewed vs interaction_count'] = df[numerical].corrwith(df['number_of_courses_viewed']).abs()['interaction_count']

# Sort and display
for key, value in sorted(results.items(), key=lambda item: item[1], reverse=True):
    print(f"{key}: {value:.4f}")

interaction_count vs annual_income: 0.0270
number_of_courses_viewed vs interaction_count: 0.0236
interaction_count vs lead_score: 0.0099
number_of_courses_viewed vs lead_score: 0.0049


In [134]:
# Question 3

# Split dataset and remove target y

from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Set the target: y
target = 'converted'

def prepare_y(df_train, df_val, df_test, target):
    y_train = df_train[target].values
    y_val = df_val[target].values
    y_test = df_test[target].values

    del df_train[target]
    del df_val[target]
    del df_test[target]

    return y_train, y_val, y_test

y_train, y_val, y_test = prepare_y(df_train, df_val, df_test, target)


In [135]:
# Determine mutual information
from sklearn.metrics import mutual_info_score

for c in categorical:
    print(f'{c}: {round(mutual_info_score(y_train, df_train[c]), 2)}')

lead_source: 0.04
industry: 0.01
employment_status: 0.01
location: 0.0


In [136]:
# Question 4

# One hot encoding categorical features

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [137]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

target_decision = (y_pred >= 0.5)

(y_val == target_decision).mean()

np.float64(0.6996587030716723)